In [1]:
import numpy as np
import matplotlib.pyplot as plt

import cbmos
import cbmos.force_functions as ff
import cbmos.solvers.euler_forward as ef
import cbmos.solvers.euler_backward as eb
import cbmos.cell as cl

In [2]:
plt.style.use('seaborn-whitegrid')
plt.style.use('tableau-colorblind10')
params = {'legend.fontsize': 'xx-large',
          'figure.figsize': (6.75, 5),
          'lines.linewidth': 3.0,
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large',
          'legend.fontsize': 'xx-large',
         'font.size': 11,
          'font.family': 'serif',
          "mathtext.fontset": "dejavuserif",
         'axes.titlepad': 12,
        'axes.labelpad': 12}
plt.rcParams.update(params)

In [3]:
# Simulation parameters
s = 1.0    # rest length
tf = 1.0  # final time
rA = 1.5   # maximum interaction distance

seed=17

params_cubic = {"mu": 5.70, "s": s, "rA": rA}
muR = 9.1
ratio = 0.21
params_poly = {'muA': muR*ratio, 'muR': muR, 'rA': rA, 'rR': 1.0/(1.0-np.sqrt(ratio)/3.0), 'n': 1.0, 'p': 1.0}
mu_gls = 1.95
params_gls = {'mu': mu_gls, 'a': 7.51}
params = {'cubic': params_cubic, 'pw. quad.': params_poly, 'GLS': params_gls}

In [4]:
force_names = ['cubic', 'pw. quad.', 'GLS']


defcolors = plt.rcParams['axes.prop_cycle'].by_key()['color']
colors = {'cubic': defcolors[0], 'pw. quad.': defcolors[5], 'GLS': defcolors[6]}

In [5]:
dim = 2
models_ef = {'pw. quad.': cbmos.CBMModel(ff.PiecewisePolynomial(), ef.solve_ivp, dim), 
             'cubic': cbmos.CBMModel(ff.Cubic(), ef.solve_ivp, dim),
             'GLS': cbmos.CBMModel(ff.Gls(), ef.solve_ivp, dim)}
models_eb = {'pw. quad.': cbmos.CBMModel(ff.PiecewisePolynomial(), eb.solve_ivp, dim), 
             'cubic': cbmos.CBMModel(ff.Cubic(), eb.solve_ivp, dim),
             'GLS': cbmos.CBMModel(ff.Gls(), eb.solve_ivp, dim)}

In [50]:
seed = 845
np.random.seed(seed) # set seed for reproducibility if necessary\n",
N = 1000
y = np.random.uniform(-10, 10, size=(N, dim))
y1 = np.random.uniform(-10, 10, size=(N*dim, ))
y2 = np.random.uniform(-10, 10, size=(N*dim, ))

In [13]:
A = models_eb['cubic'].jacobian(y_np, params['cubic'])

In [14]:
A.shape

(2000, 2000)

In [20]:
y = y_np.reshape(-1)

In [33]:
dt = 0.01

In [57]:
%time (np.eye(len(y1)) - dt*models_eb['cubic'].jacobian(y1, params['cubic'])) @ y1

CPU times: user 173 ms, sys: 22.3 ms, total: 195 ms
Wall time: 348 ms


array([ 9.03214931, -0.29017708, -0.32955059, ...,  8.64222376,
       -0.58735565, -7.91810275])

In [19]:
y_np.reshape(-1)

array([-5.04160435, -8.29309112, -4.85354749, ...,  0.07399161,
       -5.72634378, -7.01952309])

In [30]:
eta=0.01

In [26]:
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [52]:
F_curly = y1 - y2 - dt*models_eb['cubic']._ode_system({})(1, y2)

In [53]:
F_curly.shape

(2000,)

In [60]:
%time 1/eta*(y1 + eta*y1 - y2 - dt*models_eb['cubic']._ode_system({})(1, y2 + eta*y1) - F_curly)

CPU times: user 65.5 ms, sys: 0 ns, total: 65.5 ms
Wall time: 157 ms


array([ 20.04444058,  -6.13182908,  10.29177929, ..., -15.60176391,
         7.88905873, -42.43153849])